In [37]:
import os
import cv2
from tqdm import tqdm
import numpy as np
import scipy
import open3d as o3d
import read_raw_file as RRF
import preprocess as PRE
import copy


In [6]:
path = r'D:\StageE23\Data\Ete_2022\Participant11\autocorrection\Prise01'
save_path1 = path + '/intensity_removed_bg/'
save_path2 = path + '/xyz_removed_bg/'
intensity_path = path + '/intensity/'
xyz_path = path + '/xyz/'

os.makedirs(save_path1, exist_ok=True)
os.makedirs(save_path2, exist_ok=True)

for filename in tqdm(os.listdir(intensity_path)):
        index_I = filename.find('_I') + 1
        xyz_file = filename[:index_I] + 'XYZ_' + filename[index_I+2:-4] + '.raw'
        image_path = os.path.join(intensity_path, filename)
        pc_path = os.path.join(xyz_path, xyz_file)
        xyz = PRE.read_single_xyz_raw_file(pc_path)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        removed_bg_points, image_no_bg = PRE.remove_bg(xyz, image)
        
        save_image_path = os.path.join(save_path1, filename)
        cv2.imwrite(save_image_path, image_no_bg)

        pc_removed_bg = o3d.geometry.PointCloud()
        pc_removed_bg.points = o3d.utility.Vector3dVector(removed_bg_points)
        cl, ind = pc_removed_bg.remove_statistical_outlier(nb_neighbors=20,
                                                std_ratio=2.0)
        inlier_removed_bg = pc_removed_bg.select_by_index(ind)

        save_xyz_path = os.path.join(save_path2, xyz_file[:-4] + '.ply')
        o3d.io.write_point_cloud(save_xyz_path, inlier_removed_bg)

100%|██████████| 82/82 [02:19<00:00,  1.70s/it]


In [3]:
pc0 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant11\autocorrection\Prise01\xyz_removed_bg\auto_01_012915_XYZ_0.ply')
pc1 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant11\autocorrection\Prise01\xyz_removed_bg\auto_01_012917_XYZ_1.ply')
pc40 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant11\autocorrection\Prise01\xyz_removed_bg\auto_01_012973_XYZ_40.ply')
pc80 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant11\autocorrection\Prise01\xyz_removed_bg\auto_01_013030_XYZ_80.ply')

o3d.visualization.draw_geometries([pc0])

In [4]:
def crop_pc(pc):
    pc_array = np.asarray(pc.points)

    legs = np.where(pc_array[:,0] < -190)
    head = np.where(pc_array[:,0] > np.max(pc_array[:,0])-250)
    armR = np.where(pc_array[:,1] > np.max(pc_array[:,1])-110)
    armL = np.where(pc_array[:,1] < np.min(pc_array[:,1])+110)

    mask = np.ones((pc_array.shape[0],3), dtype=bool)
    mask[legs] = False
    mask[head] = False
    mask[armR] = False
    mask[armL] = False
    #mask.reshape((500123, 3))

    back = np.array(pc_array[mask], dtype=np.float64)

    pc = np.array(back.reshape((back.shape[0]//3, 3)))

    pc_cropped = o3d.geometry.PointCloud()
    pc_cropped.points = o3d.utility.Vector3dVector(pc)
    
    return pc_cropped

pc_center = crop_pc(pc0)
o3d.visualization.draw_geometries([pc_center])


In [21]:
body0 = crop_pc(pc0)
body1 = crop_pc(pc1)
body40 = crop_pc(pc40)
body80 = crop_pc(pc80)

icp1 = o3d.pipelines.registration.registration_icp(body0, body1, 0.1)
print(icp1.transformation)

icp40 = o3d.pipelines.registration.registration_icp(body0, body40, 0.1)
print(icp40.transformation)

#draw_registration_result_original_color(body0, body1, icp.transformation)

[[ 9.99999999e-01  1.75378591e-05  4.58211881e-05 -9.37219604e-02]
 [-1.75525778e-05  9.99999948e-01  3.21239022e-04 -5.64499074e-01]
 [-4.58155519e-05 -3.21239826e-04  9.99999947e-01 -4.54371724e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.99999999e-01  5.74272416e-06  4.93823243e-05 -8.81731783e-02]
 [-5.73692305e-06  9.99999993e-01 -1.17472814e-04  2.07972091e-01]
 [-4.93829985e-05  1.17472531e-04  9.99999992e-01 -5.51865912e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [44]:
mt = icp1.transformation
print(np.linalg.eig(mt)[1])

[[-9.88536980e-01+0.j         -3.77950307e-02-0.09984415j
  -3.77950307e-02+0.09984415j  9.88536980e-01+0.j        ]
 [ 1.40995002e-01+0.j          5.39070417e-03-0.70002225j
   5.39070417e-03+0.70002225j -1.40995002e-01+0.j        ]
 [-5.39911852e-02+0.j          7.06075404e-01+0.j
   7.06075404e-01-0.j          5.39911854e-02+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00-0.j          1.12238782e-13+0.j        ]]
